In [ ]:
import sys
sys.path.append('..')
import umbrella_mesh
import elastic_rods
import linkage_vis
from umbrella_mesh import UmbrellaEnergyType
from bending_validation import suppress_stdout as so
from visualization_helper import *
from matplotlib import pyplot as plt
import numpy as np

### Initial Deployment

In [ ]:
name = 'sphere_cap_0.3_one_ring'

In [ ]:
input_path = '../../data/{}.json'.format(name)
target_mesh_path = '../../data/target_meshes/{}.obj'.format('sphere_cap_0.3')

In [ ]:
#input_path = '../../data/{}.json'.format('hemisphere')
#target_mesh_path = '../../data/target_meshes/{}.obj'.format('hemisphere')

In [ ]:
from load_jsondata import read_data
input_data, io = read_data(filepath = input_path)
import mesh
target_mesh = mesh.Mesh(input_data['target_v'], input_data['target_f'])

In [ ]:
curr_um = umbrella_mesh.UmbrellaMesh(io)
thickness = io.material_params[6]
curr_um.targetDeploymentHeight = thickness

#### Pin Rigid Motion



In [ ]:
use_pin = False

In [ ]:
driver = curr_um.centralJoint()
jdo = curr_um.dofOffsetForJoint(driver)
fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + curr_um.rigidJointAngleDoFIndices()

In [ ]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-10
OPTS.verbose = 1
OPTS.beta = 1e-8
OPTS.niter = 1000
OPTS.verboseNonPosDef = False

In [ ]:
rod_colors = []
for ri in range(curr_um.numSegments()):
    rod_colors.append(np.ones(curr_um.segment(ri).rod.numVertices()) * ri)

In [ ]:
curr_um.uniformDeploymentEnergyWeight = 1e-3
curr_um.targetDeploymentHeight = thickness * 5
curr_um.repulsionEnergyWeight = 0
curr_um.attractionWeight = 0.001
curr_um.setHoldClosestPointsFixed(False)
curr_um.scaleInputPosWeights(0.5)

In [ ]:
dof = curr_um.getDoFs()
for i in range(curr_um.numJoints()):
    if (curr_um.joint(i).jointType() == umbrella_mesh.JointType.X):
        dof[curr_um.dofOffsetForJoint(i) + 6] = 1e-5
curr_um.setDoFs(dof)

In [ ]:
import fd_validation
class LAEWrapper():
    def __init__(self, um): self.obj = um
    def numVars(self):    return self.obj.numDoF()
    def getVars(self):    return self.obj.getDoFs()
    def setVars(self, x): self.obj.setDoFs(x);
    def energy(self):     return self.obj.linearActuatorEnergy()
    def gradient(self):   return self.obj.linearActuatorGradient()
    def hessian (self):   return self.obj.linearActuatorHessian()
    def hessVec(self, v): return self.obj.linearActuatorHessVec(v)

In [ ]:
lae = LAEWrapper(curr_um)

In [ ]:
lae.setVars(lae.getVars() + 1e-2 * np.random.normal(size=lae.numVars()))

In [ ]:
fd_validation.gradConvergencePlot(LAEWrapper(curr_um))

In [ ]:
fd_validation.hessConvergencePlot(LAEWrapper(curr_um))

In [ ]:
fd_validation.hessConvergencePlot(LAEWrapper(curr_um), testHessVec=True)

In [ ]:
curr_um.updateRotationParametrizations()

In [ ]:
fd_validation.hessConvergencePlot(LAEWrapper(curr_um))

In [ ]:
perturb = np.random.normal(size=lae.numVars())
joint_pos_indices = np.array(curr_um.jointPositionDoFIndices())
omega_indices = np.array(curr_um.jointPositionDoFIndices()) + 3

In [ ]:
import umbrella_finite_diff

In [ ]:
curr_um.deploymentForceType = umbrella_mesh.DeploymentForceType.LinearActuator

In [ ]:
curr_um.updateRotationParametrizations()

In [ ]:
curr_um.updateSourceFrame()

In [ ]:
import parallelism

In [ ]:
parallelism.set_max_num_tbb_threads(1)

In [ ]:
umbrella_finite_diff.gradient_convergence_plot(curr_um, umbrellaEnergyType=umbrella_mesh.UmbrellaEnergyType.Elastic)

In [ ]:
umbrella_finite_diff.gradient_convergence_plot(curr_um, umbrellaEnergyType=umbrella_mesh.UmbrellaEnergyType.Deployment)

In [ ]:
umbrella_finite_diff.hessian_convergence_plot(curr_um, umbrellaEnergyType=umbrella_mesh.UmbrellaEnergyType.Deployment)